# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [19]:
!pip install scipy==1.1.0
!pip install plot_keras_history

In [20]:
import numpy as np
import os
#from imageio import imread
from scipy.misc import imread, imresize
#from skimage.transform import resize
from imageio import imread
import datetime
import os
import warnings
import cv2


In [21]:
from plot_keras_history import plot_history
import matplotlib.pyplot as plt

We set the random seed so that the results don't vary drastically.

In [22]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30) 

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [23]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
train_doc = np.random.permutation(open('/content/gdrive/MyDrive/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/gdrive/MyDrive/Project_data/val.csv').readlines())


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [26]:
def generator(source_path, folder_list, batch_size,y,z,augment):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,7,15,21,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)
        remaining_batch_size=int(len(t)/batch_size)%batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,5,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    height, width , channel = image.shape
                    start_w=((width-y)-20//2)
                    end_w=(((width-y)+10//2)+width)
                    start_h=(((height-z)-20//2))
                    end_h=(((height-z)+10//2)+height)
                    #cropped=shifted[start_w:end_w,start_h:end_h,:]
                    image_resized=imresize(image,(y,z,3))


                    batch_data[folder,idx,:,:,0] = image_resized[:,:,0]/255
                    batch_data[folder,idx,:,:,1] = image_resized[:,:,1]/255
                    batch_data[folder,idx,:,:,2] = image_resized[:,:,2]/255

                    batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

                    yield batch_data, batch_labels
                      

        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,5,y,z,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    height, width , channel = image.shape
                    start_w=((width-y)//2)
                    end_w=(((width-y)//2)+width)
                    start_h=(((height-z)//2))
                    end_h=(((height-z)//2)+height)
                    #cropped=shifted[start_w:end_w,start_h:end_h,:]
                    image_resized=imresize(image,(y,z,3))



                    batch_data[folder,idx,:,:,0] = image_resized[:,:,0]/255
                    batch_data[folder,idx,:,:,1] = image_resized[:,:,1]/255
                    batch_data[folder,idx,:,:,2] = image_resized[:,:,2]/255

                    batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

                    yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [27]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/gdrive/MyDrive/Project_data/train'
val_path = '/content/gdrive/MyDrive/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [28]:
'''
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D
from tensorflow.keras.utils import to_categorical
import h5py
import numpy as np
import matplotlib.pyplot as plt'''

'\nimport tensorflow\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D\nfrom tensorflow.keras.utils import to_categorical\nimport h5py\nimport numpy as np\nimport matplotlib.pyplot as plt'

In [29]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
def conv_model(y,z):
  model = Sequential()
  model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(5,y,z,3)))
  model.add(BatchNormalization())
  model.add(Activation('elu'))
  model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
  '''
  model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('elu'))
  model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

  '''
  model.add(Flatten())
  model.add(Dropout(0.2))
  model.add(Dense(512, activation='elu'))
  model.add(Dropout(0.2))
  model.add(Dense(5, activation='softmax'))
  return model


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [30]:
model=conv_model(90,90)

In [31]:
sgd = tf.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 5, 90, 90, 64)     5248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 90, 90, 64)     256       
_________________________________________________________________
activation_1 (Activation)    (None, 5, 90, 90, 64)     0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 2, 45, 90, 64)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 518400)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 518400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [32]:
def train_generator(train_path, train_doc, batch_size,y,z):
  return generator(train_path, train_doc, batch_size,y,z,augment=True)
def val_generator(val_path, val_doc, batch_size,y,z):
  return generator(val_path, val_doc, batch_size, y,z,augment=False)

In [33]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
filepath = model_name + 'best.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

In [34]:
'''model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]'''

"model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'\n    \nif not os.path.exists(model_name):\n    os.mkdir(model_name)\n        \nfilepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'\n\ncheckpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)\n\nLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)\ncallbacks_list = [checkpoint, LR]"

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [35]:
def calc():
  if (num_train_sequences%batch_size) == 0:
      steps_per_epoch = int(num_train_sequences/batch_size)
  else:
      steps_per_epoch = (num_train_sequences//batch_size) + 1

  if (num_val_sequences%batch_size) == 0:
      validation_steps = int(num_val_sequences/batch_size)
  else:
      validation_steps = (num_val_sequences//batch_size) + 1
  return steps_per_epoch,validation_steps

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
batch_size=32
num_epochs=10
y=90
z=90
steps_per_epoch,validation_steps=calc()
history=model.fit(train_generator(train_path, train_doc, batch_size=batch_size, y=y,z=z), steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator(val_path, val_doc, batch_size=batch_size, y=y,z=z), 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/gdrive/MyDrive/Project_data/train ; batch size = 32
Epoch 1/10
21/21 [==============================] - 7s 308ms/step - loss: 1.4341 - categorical_accuracy: 0.1920 - val_loss: 262.6766 - val_categorical_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 262.67661, saving model to model_init_2021-10-2310_37_20.794942/best.h5


In [ ]:

plot_history(history, single_graphs=True)
plt.close()

### RNN + CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,TimeDistributed,GRU,Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
cnn =Sequential([resnet])
cnn.add(Conv2D(64,(2,2),strides=(1,1)))
cnn.add(Conv2D(16,(3,3),strides=(1,1)))
cnn.add(Flatten()) 

In [ ]:
model2 = Sequential()
model2.add(TimeDistributed(cnn,input_shape=(5,224,224,3)))
model2.add(GRU(16,input_shape=(None,30,256),return_sequences=True))
model2.add(GRU(8))
model2.add(Dense(5,activation='softmax'))

In [ ]:
model2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 5, 256)            24121296  
_________________________________________________________________
gru_2 (GRU)                  (None, 5, 16)             13152     
_________________________________________________________________
gru_3 (GRU)                  (None, 8)                 624       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 45        
Total params: 24,135,117
Trainable params: 24,081,997
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 64)          524352    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          9232      
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
Total params: 24,121,296
Trainable params: 24,068,176
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.001)
model2.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
'''history=model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)'''

'history=model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, \n                    callbacks=callbacks_list, validation_data=val_generator, \n                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)'